In [2]:
关于本次征途杯瑕疵检测算法以及模型的部分说明：
队伍：NK-CV-Club

编程思路：
复赛中的任务是在非平坦区域中的瑕疵检测。具体而言，是在背景图片的颜色和形状较之初赛多样化复杂化的情况下，
在其中标注出所有瑕疵所在的像素。考虑到背景非常复杂，使用无监督的训练方法可能无法取得非常良好的效果，
因此，我们使用了有监督的图像分割的办法，通过考察瑕疵的形状以及颜色，使用程序在线地在好品图上生成瑕疵与对应的标签，
并将其输入模型进行训练。考虑到复赛中的瑕疵大小各异，因此我们选用了擅长应对多尺度分割的res2net作为我们模型的主体，
同时使用基于res2net的csf作为我们的模型。loss函数使用的是交叉熵函数。
除此之外，在模型的瑕疵生成方面，为了让生成的瑕疵更加逼真，我们使用了高斯模糊和图片叠加的办法，
让生成的瑕疵更像是真实的瑕疵。考虑到part1和part2的图片有较为严重的噪点现象，高斯模糊也可以针对这一点进行改善。

核心代码之瑕疵生成：
def get_random_shape():
    out = []
    black_type = random.randint(0, 9)
    if black_type == 0:
        src_tmp = np.array(np.zeros((128, 128)), dtype='uint8')
        point_num = 10
        point_list = []
        left_x = random.randint(0, 127)
        left_y = random.randint(0, 127)
        for i in range(point_num):
            center_x = random.randint(0, 5)
            center_y = random.randint(0, 5)
            point_list.append((left_x + center_x, left_y + center_y))
        point_list = np.array(point_list)
        hull = cv.convexHull(point_list)
        # print(hull)
        src_tmp = cv.drawContours(src_tmp, [hull], 0, 255, -1)
        src_tmp = rotato(src_tmp, random.randint(0, 360), 1, hull[0][0])
        out = src_tmp > 125
        return np.where(out, 1.0, 0)
    #中略
    elif black_type == 3 or black_type == 6:
        src_tmp = np.array(np.zeros((128, 128)), dtype='uint8')
        point_num = 10
        point_list = []
        left_x = random.randint(0, 127)
        left_y = random.randint(0, 127)
        for i in range(point_num):
            center_x = random.randint(0, 30)
            center_y = random.randint(0, 1)
            point_list.append((left_x + center_x, left_y + center_y))
        point_list = np.array(point_list)
        hull = cv.convexHull(point_list)
        # print(hull)
        src_tmp = cv.drawContours(src_tmp, [hull], 0, 255, -1)
        src_tmp = rotato(src_tmp, random.randint(0, 360), 1, hull[0][0])
        out = src_tmp > 125
        return np.where(out, 1.0, 0)
    elif black_type == 4:
        src_tmp = np.array(np.zeros((128, 128)), dtype='uint8')
        left_x = random.randint(0, 50)
        left_y = random.randint(0, 50)
        right_x = random.randint(80, 127)
        right_y = random.randint(80, 127)
        src_tmp = cv.line(src_tmp, (left_x, left_y), (right_x, right_y), 255, random.randint(1, 2), lineType=cv.LINE_AA)
        src_tmp = rotato(src_tmp, random.randint(0, 360), 1, (64, 64))
        out = src_tmp > 125
        return np.where(out, 1.0, 0)
    elif black_type == 5 or black_type == 7:
        n = random.randint(3, 5)
        r = 0.7
        N = n * 3 + 1  # number of points in the Path
        # There is the initial point and 3 points per cubic bezier curve. Thus, the curve will only pass though n points, which will be the sharp edges, the other 2 modify the shape of the bezier curve

        angles = np.linspace(0, 2 * np.pi, N)
        codes = np.full(N, Path.CURVE4)
        codes[0] = Path.MOVETO

        verts = np.stack((np.cos(angles), np.sin(angles))).T * (2 * r * np.random.random(N) + 1 - r)[:, None]
        verts[-1, :] = verts[0, :]  # Using this instad of Path.CLOSEPOLY avoids an innecessary straight line
        path = Path(verts, codes)

        fig = plt.figure()
        ax = fig.add_subplot(111)
        patch = patches.PathPatch(path, facecolor='none', lw=2)
        ax.add_patch(patch)

        ax.set_xlim(np.min(verts) * 1.1, np.max(verts) * 1.1)
        ax.set_ylim(np.min(verts) * 1.1, np.max(verts) * 1.1)
        ax.axis('off')  # removes the axis to leave only the shape
        plt.savefig('./1.png')
        plt.close()
        img = cv.imread('1.png', 0)
        try:
            _, c, h = cv.findContours(img, cv.RETR_TREE, cv.CHAIN_APPROX_NONE)
        except:
            c, h = cv.findContours(img, cv.RETR_TREE, cv.CHAIN_APPROX_NONE)
        temp = np.zeros(img.shape, np.uint8) * 255

        cv.drawContours(temp, c, 1, 255, -1)
        temp = cv.resize(temp, (128, 128))
        temp = np.where(temp > 128, get_weight((64, 64)), 0.)
        src_tmp = rotato(temp, random.randint(0, 360), random.randint(4, 8) / 10,
                         center=(random.randint(30, 100), random.randint(30, 100)))
        return src_tmp

核心代码之Bottle2neck：
class Bottle2neck(nn.Module):
    expansion = 4

    def __init__(self, inplanes, planes, stride=1, dilation_=1, downsample=None, baseWidth=26, scale=4, stype='normal'):
        """ Constructor
        Args:
            inplanes: input channel dimensionality
            planes: output channel dimensionality
            stride: conv stride. Replaces pooling layer.
            downsample: None when stride = 1
            baseWidth: basic width of conv3x3
            scale: number of scale.
            type: 'normal': normal set. 'stage': first block of a new stage.
        """
        super(Bottle2neck, self).__init__()

        width = int(math.floor(planes * (baseWidth / 64.0)))
        self.conv1 = nn.Conv2d(inplanes, width * scale, kernel_size=1, bias=False)
        self.bn1 = nn.BatchNorm2d(width * scale, affine=affine_par)
        for i in self.bn1.parameters():
            i.requires_grad = False
        if scale == 1:
            self.nums = 1
        else:
            self.nums = scale - 1
        if stype == 'stage':
            self.pool = nn.AvgPool2d(kernel_size=3, stride=stride, padding=1)
        convs = []
        bns = []
        for i in range(self.nums):
            convs.append(nn.Conv2d(width, width, kernel_size=3, stride=stride, dilation=dilation_, padding=dilation_,
                                   bias=False))
            bns.append(nn.BatchNorm2d(width))
        self.convs = nn.ModuleList(convs)
        self.bns = nn.ModuleList(bns)
        for j in range(self.nums):
            for i in self.bns[j].parameters():
                i.requires_grad = False

        self.conv3 = nn.Conv2d(width * scale, planes * self.expansion, kernel_size=1, bias=False)
        self.bn3 = nn.BatchNorm2d(planes * self.expansion)
        for i in self.bn3.parameters():
            i.requires_grad = False
        self.relu = nn.ReLU(inplace=True)
        self.downsample = downsample
        self.stype = stype
        self.scale = scale
        self.width = width

    def forward(self, x):
        residual = x

        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)

        spx = torch.split(out, self.width, 1)
        for i in range(self.nums):
            if i == 0 or self.stype == 'stage':
                sp = spx[i]
            else:
                sp = sp + spx[i]
            sp = self.convs[i](sp)
            sp = self.relu(self.bns[i](sp))
            if i == 0:
                out = sp
            else:
                out = torch.cat((out, sp), 1)
        if self.scale != 1 and self.stype == 'normal':
            out = torch.cat((out, spx[self.nums]), 1)
        elif self.scale != 1 and self.stype == 'stage':
            out = torch.cat((out, self.pool(spx[self.nums])), 1)

        out = self.conv3(out)
        out = self.bn3(out)

        if self.downsample is not None:
            residual = self.downsample(x)

        out += residual
        out = self.relu(out)

        return out

核心代码之csfres2net：
class Res2Net(nn.Module):
    def __init__(self, block, layers, baseWidth=26, scale=4, input_channle=3, need_pretrain=False):
        self.inplanes = 64
        super(Res2Net, self).__init__()
        self.baseWidth = baseWidth
        self.scale = scale
        self.conv1 = nn.Sequential(
            nn.Conv2d(input_channle, 32, 3, 2, 1, bias=False),
            nn.BatchNorm2d(32),
            nn.ReLU(inplace=True),
            nn.Conv2d(32, 32, 3, 1, 1, bias=False),
            nn.BatchNorm2d(32),
            nn.ReLU(inplace=True),
            nn.Conv2d(32, 64, 3, 1, 1, bias=False)
        )
        self.bn1 = nn.BatchNorm2d(64, affine=affine_par)
        for i in self.bn1.parameters():
            i.requires_grad = False
        self.relu = nn.ReLU(inplace=True)
        self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
        self.layer1 = self._make_layer(block, 64, layers[0])
        self.layer2 = self._make_layer(block, 128, layers[1], stride=2)
        self.layer3 = self._make_layer(block, 256, layers[2], stride=2)
        self.layer4 = self._make_layer(block, 512, layers[3], stride=2)
        self.avgpool = nn.AvgPool2d(7, stride=1)
        if need_pretrain:
            self.fc = nn.Linear(512 * block.expansion, 1000)

        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.kaiming_normal_(m.weight, mode='fan_out', nonlinearity='relu')
            elif isinstance(m, nn.BatchNorm2d):
                nn.init.constant_(m.weight, 1)
                nn.init.constant_(m.bias, 0)
        
    #forward略

class MSBlock(nn.Module):
    def __init__(self, in_channels, out_channels, dilations=[1, 2, 4, 8, 16]):
        super(MSBlock, self).__init__()
        self.dilations = dilations
        each_out_channels = out_channels // 5
        self.msconv = nn.ModuleList()
        for i in range(len(dilations)):
            if i != len(dilations) - 1:
                this_outc = each_out_channels
            else:
                this_outc = out_channels - each_out_channels * (len(dilations) - 1)
            self.msconv.append(
                nn.Conv2d(in_channels, this_outc, 3, padding=dilations[i], dilation=dilations[i], bias=False))
        self.bn = nn.GroupNorm(32, out_channels)
        self.prelu = nn.PReLU(out_channels)

    def forward(self, x):
        outs = []
        for i in range(len(self.dilations)):
            outs.append(self.msconv[i](x))
        out = torch.cat(outs, dim=1)
        del outs
        out = self.prelu(self.bn(out))
        return out


class CSFNet(nn.Module):
    def __init__(self, num_classes=1, input_channle=3, need_pretrain=None):
        super(CSFNet, self).__init__()
        self.base = Res2Net(Bottle2neck, [3, 4, 6, 3], baseWidth=26, scale=4, input_channle=input_channle,
                            need_pretrain=(need_pretrain is not None))
        if need_pretrain is not None:
            self.base.load_state_dict(torch.load(need_pretrain))

        fuse_in_channel = 256 + 512 + 1024 + 2048
        fuse_in_split = [1 / 15, 2 / 15, 4 / 15, 8 / 15]
        fuse_out_channel = 128 + 256 + 512 + 512
        fuse_out_split = [1 / 11, 2 / 11, 4 / 11, 4 / 11]

        self.fuse = gOctaveCBR(fuse_in_channel, fuse_out_channel, kernel_size=(1, 1), padding=0,
                               alpha_in=fuse_in_split, alpha_out=fuse_out_split, stride=1)
        self.ms = PallMSBlock(fuse_out_channel, fuse_out_channel, alpha=fuse_out_split)
        self.fuse1x1 = gOctaveCBR(fuse_out_channel, fuse_out_channel, kernel_size=(1, 1), padding=0,
                                  alpha_in=fuse_out_split, alpha_out=[1, ], stride=1)
        self.cls_layer = nn.Conv2d(fuse_out_channel, num_classes, kernel_size=1)

    def forward(self, x):
        features = self.base(x)
        fuse = self.fuse(features)
        fuse = self.ms(fuse)
        fuse = self.fuse1x1(fuse)
        output = self.cls_layer(fuse[0])
        output = F.interpolate(output, x.size()[2:], mode='bilinear', align_corners=False)

        return output

SyntaxError: invalid character in identifier (<ipython-input-2-ace6d2158950>, line 1)

In [3]:
特色亮点：
1，使用了监督学习方法，运用神经网络强大的拟合能力做到了端到端处理，超参数量较少。同时性能超越一般的基于图像生成的无监督学习方法。
2，使用了在多尺度方面表现相当优秀的res2net作为基础。经过我们的实验，基于res2net的csf对比起同样是进行语义分割任务的FCN，deeplab模型来说，
在漏检方面有更好的表现。换言之，res2net可以把面积稍大的瑕疵全部检出，而FCN和deeplab等模型只能检测出来部分，有相当严重的漏检情况。
3，在生成瑕疵时，为了让生成的瑕疵更加接近真实的瑕疵，对于不规则的瑕疵，我们使用了贝塞尔曲线来生成不规则的瑕疵，在生成之后，以图形的中心为原点生成了高斯核作为瑕疵的权值，
并以这个权值作为图片叠加的权值，实现了中间浓，边缘浅的瑕疵。对于规则的瑕疵，我们使用了随机描点，然后用findContours和drawContours染色的办法来实现。

SyntaxError: invalid character in identifier (<ipython-input-3-4025dc93eee8>, line 1)

In [ ]:
编程语言：

Python==3.6 Pytorch==1.2.0

软件运行环境：

Linux python==3.6 pytorch==1.2.0 torchvision==0.7.0 opencv-python==4.1.2.30 opencv-contrib-python==3.4.2.17 pathLib2==2.3.5 matplotlib==3.1.1

硬件配置环境：

内存至少8G
显卡：2080ti
CPU： 9900kf
整体训练+推测需要11小时
在硬件环境改变的时候，运行时间随之改变

运行说明：

cd到code部分运行main.py即可在result中获得结果。
运行细节如下：
- main.py中首先调用瑕疵生成器gen1,gen2,gen3,这三个瑕疵生成器分别从OK大图中截取小图生成瑕疵并保存文件
- 生成完成之后调用网络训练代码train_part1,train_part2,train_part3,这三个部分分别使用语义分割模块进行训练
- 训练过程中会存储ckpt，只有调用p1,p2,p3,这3个模块进行端到端推测，得到初步的json结果
- 最后一步，因为初步json结果可能存在过检或者瑕疵面积生成过小等问题，程序会调用filter代码进行导向滤波和形态学膨胀，最后输出最终json，压缩后放入result
备注信息：

在part3的模型的训练过程中，我们使用了预训练的模型为基础进行微调，以下是预训练模型的网址：
https://shanghuagao.oss-cn-beijing.aliyuncs.com/res2net/res2net50_v1b_26w_4s-3cf99910.pth